In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from backend.extractor.utils import parse_html, save_html
from backend.fetcher.fetcher import fetch_html, fetch_html_request
from backend.extractor.utils import clean_html, prepare_html, traverse_and_modify
from backend.extractor.task.container_extractor import ContainerExtractor
from backend.fetcher.fetcher import fetch_html, fetch_html_request
import asyncio
from bs4 import BeautifulSoup

In [3]:
def load_html_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        html = file.read()
    return html

def clean_html(html):
    soup = BeautifulSoup(html, "html.parser")

    # Example: Remove all <script> and <style> tags
    for script_or_style in soup(["script", "style", "link"]):
        script_or_style.decompose()

    # Additional cleaning can be done here by removing other unwanted tags

    return str(soup)


def save_html_to_file(html, filename):
    with open(filename, "w", encoding="utf-8") as file:
        file.write(html)
    print(f"HTML content saved to {filename}")

In [4]:
from backend.fetcher.fetcher import fetch_infinite_page

see_more = await fetch_infinite_page(
    "https://www.facebook.com/HcmIUcfs/posts/pfbid0zgoU5Cmy4gjtWosc7LbL21naQdGpqaJdzFPKwEMf6ao8uGGFnjrUiFjmGMDm9Pusl",
    max_duration=50,
    expand_button_text=["View all", "View more comments, View all (\d+) comments", "Xem tất cả (\d+) phản hồi", "Xem (\d+) phản hồi", "Xem thêm"],
)
cleaned_html_with_see_more = clean_html(see_more)
save_html_to_file(cleaned_html_with_see_more, "html/post.html")

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/mp/m1gx2csd0h5f6ndzf6xtmn_40000gp/T/ipykernel_1090/1317977172.py:6: SyntaxWarning: invalid escape sequence '\d'
  expand_button_text=["View all", "View more comments, View all (\d+) comments", "Xem tất cả (\d+) phản hồi", "Xem (\d+) phản hồi", "Xem thêm"],
/var/folders/mp/m1gx2csd0h5f6ndzf6xtmn_40000gp/T/ipykernel_1090/1317977172.py:6: SyntaxWarning: invalid escape sequence '\d'
  expand_button_text=["View all", "View more comments, View all (\d+) comments", "Xem tất cả (\d+) phản hồi", "Xem (\d+) phản hồi", "Xem thêm"],
/var/folders/mp/m1gx2csd0h5f6ndzf6xtmn_40000gp/T/ipykernel_1090/1317977172.py:6: SyntaxWarning: invalid escape sequence '\d'
  expand_button_text=["View all", "View more comments, View all (\d+) comments", "Xem tất cả (\d+) phản hồi", "Xem (\d+) phản hồi", "Xem thêm"],
2024-06-05 01:03:42,367 - fetcher.py: fe

HTML content saved to html/post.html


In [59]:
cleaned_html = prepare_html(cleaned_html_with_see_more)

2024-06-04 16:18:22,718 - utils.py: parse_html(): 79:	Parsing HTML string to BeautifulSoup object
--------
2024-06-04 16:18:23,523 - utils.py: clean_html(): 16:	Cleaning HTML, removing all path that does not contains navigable string
--------


[document] -> html -> head -> title -> IU Confessions - #IU_Cfs11567 31/05/2024 9:55:13Em chào... | Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> div -> a -> svg -> title -> Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> a -> div -> div -> span -> span -> Đăng nhập
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> div -> div -> div -> div -> div -> span -> span -> Đăng nhập
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> div -> span -> a -> Bạn quên tài khoản ư?
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> blockquote -> span -> span -> Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> d

In [65]:
example_container = {
    "timestamp": "31/05/2024 9:55:13",
    "post": "Em chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường về nhiều mặt ở mức độ khá xấu làm em có hơi hoang mang và lo sợ nên em muốn nghe thêm lời khuyên của các anh chị trong trường để biết liệu có nên học hay không ạ. Em cũng muốn hỏi là tất cả các chương trình giảng dạy của trường đều dạy bằng tiếng anh thì lúc đậu rồi có cần phải kiểm tra trình độ tiếng anh hay gì không ạ do em không có bằng ngôn ngữ gì hết;;-;;. Cảm ơn các anh chị đã đọc bài & đưa ra ý kiến giúp em ạ",
}

In [66]:
extractor = ContainerExtractor(cleaned_html, example_container)

2024-06-04 16:26:50,860 - utils.py: clean_html(): 16:	Cleaning HTML, removing all path that does not contains navigable string
--------


[document] -> html -> head -> title -> IU Confessions - #IU_Cfs11567 31/05/2024 9:55:13Em chào... | Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> div -> a -> svg -> title -> Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> a -> div -> div -> span -> span -> Đăng nhập
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> div -> div -> div -> div -> div -> span -> span -> Đăng nhập
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> form -> div -> div -> span -> a -> Bạn quên tài khoản ư?
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> div -> blockquote -> span -> span -> Facebook
[document] -> html -> body -> div -> div -> div -> div -> div -> div -> div -> div -> d

In [67]:
structured_extracted_contents, posts = await extractor.container_extract_run_task()

2024-06-04 16:26:53,148 - container_extractor.py: prepare_extract(): 41:	Preparing extract
--------
2024-06-04 16:26:53,149 - extractor.py: find_string_tag(): 16:	Found string in soup at: <title>IU Confessions - #IU_Cfs11567 
    31/05/2024 9:55:13
    Em chÃ o... | Facebook</title>
--------
2024-06-04 16:26:53,150 - extractor.py: find_string_tag(): 16:	Found string in soup at: <div dir="auto" style="text-align: start;">Em chÃ o cÃ¡c anh chá» áº¡. Chuyá»n lÃ  em sáº¯p thi TNTHPT vÃ  cÃ³ nguyá»n vá»ng muá»n vÃ o trÆ°á»ng IU áº¡. Em cÃ³ Äá»c qua nhiá»u bÃ i cfs vÃ  ná»i dung cÃ¡c bÃ i ÄÃ³ Äa pháº§n ÄÃ¡nh giÃ¡ trÆ°á»ng vá» nhiá»u máº·t á» má»©c Äá» khÃ¡ xáº¥u lÃ m em cÃ³ hÆ¡i hoang mang vÃ  lo sá»£ nÃªn em muá»n nghe thÃªm lá»i khuyÃªn cá»§a cÃ¡c anh chá» trong trÆ°á»ng Äá» biáº¿t liá»u cÃ³ nÃªn há»c hay khÃ´ng áº¡. Em cÅ©ng muá»n há»i lÃ  táº¥t cáº£ cÃ¡c chÆ°Æ¡ng trÃ¬nh giáº£ng dáº¡y cá»§a trÆ°á»ng Äá»u dáº¡y báº±ng tiáº¿ng anh thÃ¬ lÃºc Äáº­u rá»i cÃ³ c

{'timestamp': ['IU Confessions - #IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào... | Facebook'], 'post': ['#IU_Cfs11567', '31/05/2024 9:55:13', 'Em chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường về nhiều mặt ở mức độ khá xấu làm em có hơi hoang mang và lo sợ nên em muốn nghe thêm lời khuyên của các anh chị trong trường để biết liệu có nên học hay không ạ. Em cũng muốn hỏi là tất cả các chương trình giảng dạy của trường đều dạy bằng tiếng anh thì lúc đậu rồi có cần phải kiểm tra trình độ tiếng anh hay gì không ạ do em không có bằng ngôn ngữ gì hết;;-;;. Cảm ơn các anh chị đã đọc bài & đưa ra ý kiến giúp em ạ', '#IU_Cfs11568', '31/05/2024 9:51:21', 'Em sn 2007, sắp thi tốt nghiệp, cảm ơn các anh chị sinh viên đã review về IU, em sẽ chuyển nguyện vọng qua UEH hoặc FPT.', '#IU_Cfs11569', '29/05/2024 1:31:36', 'Em mới đầu rất thích trường cũng mơ mộng khi vào trường. ( vì em nghe 

In [ ]:
for i, post in enumerate(structured_extracted_contents):
    if (post["tag"] == "IU Confessions") or (
        not post["content"] and not post["content2"]
    ):
        structured_extracted_contents.pop(i)

In [48]:
import re


pattern = re.compile(r"(Expand|See more|Xem thêm|Hiển thị|View all (\d+) comments)")
text = "View all 50 comments"

if re.search(pattern, text):
    print("Match found")
else:
    print("No match")

Match found


In [46]:
button_text = ["View all", "See more", "HI"]

print("|".join(button_text))

View all|See more|HI


In [5]:
import re
remove_line = re.sub(r"(\r\n|\r|\n){2,}", lambda x: "\n", cleaned_html_with_see_more)

In [9]:
cleaned_html_with_see_more

'<!DOCTYPE html>\n<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="vi"><head><meta content="noarchive" name="bingbot"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs..." name="description"/><meta content="video.other" property="og:type"/><meta content="IU Confessions" property="og:title"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường..." property="og:description"/><meta content="https://www.facebook.com/HcmIUcfs/posts/iu_cfs11567-31052024-95513em-ch%C3%A0o-c%C3%A1c-anh-ch%E1%BB%8B-%E1%BA%A1-chuy%E1%BB%87n-l%C3%A0-em-s%E1%BA%AFp-thi-tnthpt-v%C3%A0-c/894616432709339/" property="og:url"

In [8]:
remove_line

'<!DOCTYPE html>\n<html class="_9dls __fb-light-mode" dir="ltr" id="facebook" lang="vi"><head><meta content="noarchive" name="bingbot"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs..." name="description"/><meta content="video.other" property="og:type"/><meta content="IU Confessions" property="og:title"/><meta content="#IU_Cfs11567 \n31/05/2024 9:55:13\nEm chào các anh chị ạ. Chuyện là em sắp thi TNTHPT và có nguyện vọng muốn vào trường IU ạ. Em có đọc qua nhiều bài cfs và nội dung các bài đó đa phần đánh giá trường..." property="og:description"/><meta content="https://www.facebook.com/HcmIUcfs/posts/iu_cfs11567-31052024-95513em-ch%C3%A0o-c%C3%A1c-anh-ch%E1%BB%8B-%E1%BA%A1-chuy%E1%BB%87n-l%C3%A0-em-s%E1%BA%AFp-thi-tnthpt-v%C3%A0-c/894616432709339/" property="og:url"

In [10]:
expand_text = "Show more, See more, Expand"
expand_button_text = expand_text.split(", ")
expand_button_text

['Show more', 'See more', 'Expand']